# 銅価格時系列分析（完全版）

Bloomberg APIから取得した銅価格データの時系列分析を行います。

## 分析内容
- LME, SHFE, CMXの銅価格推移
- 各取引所の価格相関分析
- ボラティリティ分析
- 移動平均線との比較
- 限月別価格分析
- 総合サマリー

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 設定
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# パス設定
import sys
import os
current_dir = os.getcwd()
config_path = os.path.join(os.path.dirname(current_dir), 'config')
sys.path.insert(0, config_path)

print(f"Config file path: {config_path}")

try:
    from data_utils import DataFetcher
    from db_config import VISUALIZATION_CONFIG
    print("SUCCESS: Module import successful")
    
    fetcher = DataFetcher()
    print("\nTesting database connection...")
    if fetcher.test_connection():
        print("SUCCESS: Database connection ready")
    else:
        print("ERROR: Database connection issue")
    
except ImportError as e:
    print(f"ERROR: Import error: {e}")
    raise ImportError("Cannot import required modules")

print("\n" + "=" * 60)
print("  COPPER PRICE COMPREHENSIVE ANALYSIS")
print("=" * 60)

## 1. データ取得

In [ ]:
print("📥 データ取得中...")

try:
    copper_prices = fetcher.get_copper_prices(days=365)
    
    if copper_prices.empty:
        raise ValueError("No copper price data found")
    
    print(f"✅ 取得完了: {len(copper_prices)} レコード")
    print(f"📅 期間: {copper_prices['TradeDate'].min()} ～ {copper_prices['TradeDate'].max()}")
    print(f"🏢 取引所: {copper_prices['ExchangeCode'].unique()}")
    print(f"📈 限月タイプ: {copper_prices['TenorTypeName'].unique()}")
    
    display(copper_prices.head())
    
except Exception as e:
    print(f"❌ エラー: {e}")
    copper_prices = pd.DataFrame()

## 2. データ前処理

In [ ]:
# 現物価格（Cash）のみを抽出
cash_prices = copper_prices[copper_prices['TenorTypeName'] == 'Cash'].copy()

if cash_prices.empty:
    print("⚠️ Cash価格データが見つかりません。全データを使用します。")
    cash_prices = copper_prices.copy()

# 取引所別にピボット
price_pivot = cash_prices.pivot_table(
    index='TradeDate',
    columns='ExchangeCode',
    values='LastPrice',
    aggfunc='first'
)

# 前方補完を使用
price_pivot = price_pivot.ffill()

print(f"📊 現物価格データ: {len(price_pivot)} 日間")
print(f"🏢 取引所: {list(price_pivot.columns)}")
if len(price_pivot) > 0:
    print(f"🗓️ 最新データ: {price_pivot.index.max()}")

# 統計サマリー
if not price_pivot.empty:
    display(price_pivot.describe())

## 3. 価格推移の可視化

In [ ]:
# Matplotlib による静的グラフ
if not price_pivot.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # 価格推移
    for exchange in price_pivot.columns:
        ax1.plot(price_pivot.index, price_pivot[exchange], 
                 label=f'{exchange} Copper Cash', linewidth=2)

    ax1.set_title('🔸 銅現物価格推移（取引所別）', fontsize=16, fontweight='bold')
    ax1.set_xlabel('日付')
    ax1.set_ylabel('価格 (USD/MT)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 日次変動率
    returns = price_pivot.pct_change().dropna()
    
    if not returns.empty:
        for exchange in returns.columns:
            ax2.plot(returns.index, returns[exchange] * 100, 
                     label=f'{exchange} Daily Return', alpha=0.7)

        ax2.set_title('🔸 日次変動率（%）', fontsize=16, fontweight='bold')
        ax2.set_xlabel('日付')
        ax2.set_ylabel('変動率 (%)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 価格データがないため、グラフを表示できません。")

In [ ]:
# Plotly による動的グラフ
if not price_pivot.empty:
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('銅現物価格推移', '日次変動率'),
        vertical_spacing=0.1
    )

    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    
    # 価格推移
    for i, exchange in enumerate(price_pivot.columns):
        fig.add_trace(
            go.Scatter(
                x=price_pivot.index,
                y=price_pivot[exchange],
                mode='lines',
                name=f'{exchange} Cash',
                line=dict(color=colors[i % len(colors)], width=2)
            ),
            row=1, col=1
        )

    # 日次変動率
    if 'returns' in locals() and not returns.empty:
        for i, exchange in enumerate(returns.columns):
            fig.add_trace(
                go.Scatter(
                    x=returns.index,
                    y=returns[exchange] * 100,
                    mode='lines',
                    name=f'{exchange} Return',
                    line=dict(color=colors[i % len(colors)], width=1),
                    opacity=0.7
                ),
                row=2, col=1
            )

    fig.update_layout(
        height=600,
        title_text="🔸 銅価格分析ダッシュボード",
        title_x=0.5,
        showlegend=True
    )

    fig.update_xaxes(title_text="日付", row=2, col=1)
    fig.update_yaxes(title_text="価格 (USD/MT)", row=1, col=1)
    fig.update_yaxes(title_text="変動率 (%)", row=2, col=1)

    fig.show()

## 4. 相関分析

In [ ]:
# 価格相関分析
if not price_pivot.empty and len(price_pivot.columns) > 1:
    # 相関計算
    price_corr = price_pivot.corr()
    returns_corr = returns.corr() if 'returns' in locals() and not returns.empty else pd.DataFrame()

    # 相関マトリックスの可視化
    if not returns_corr.empty:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

        # 価格相関
        sns.heatmap(price_corr, annot=True, cmap='coolwarm', center=0, 
                    square=True, ax=ax1, cbar_kws={'label': '相関係数'})
        ax1.set_title('🔸 価格相関マトリックス', fontsize=14, fontweight='bold')

        # リターン相関
        sns.heatmap(returns_corr, annot=True, cmap='coolwarm', center=0, 
                    square=True, ax=ax2, cbar_kws={'label': '相関係数'})
        ax2.set_title('🔸 リターン相関マトリックス', fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # 相関分析結果の表示
        print("📊 価格相関分析結果:")
        print(price_corr)
        print("\n📊 リターン相関分析結果:")
        print(returns_corr)
    else:
        print("⚠️ 相関分析用のデータが不足しています。")
else:
    print("⚠️ 相関分析に十分なデータがありません。")

## 5. ボラティリティ分析

In [ ]:
# 30日移動ボラティリティ計算
if 'returns' in locals() and not returns.empty:
    rolling_vol = returns.rolling(window=30).std() * np.sqrt(252) * 100  # 年率換算

    # ボラティリティ推移の可視化
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # 30日移動ボラティリティ
    for exchange in rolling_vol.columns:
        ax1.plot(rolling_vol.index, rolling_vol[exchange], 
                 label=f'{exchange} 30日移動ボラティリティ', linewidth=2)

    ax1.set_title('🔸 30日移動ボラティリティ（年率%）', fontsize=16, fontweight='bold')
    ax1.set_xlabel('日付')
    ax1.set_ylabel('ボラティリティ (%)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # ボラティリティ分布
    rolling_vol.dropna().hist(bins=30, alpha=0.7, ax=ax2)
    ax2.set_title('🔸 ボラティリティ分布', fontsize=16, fontweight='bold')
    ax2.set_xlabel('ボラティリティ (%)')
    ax2.set_ylabel('頻度')
    ax2.legend(rolling_vol.columns)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # ボラティリティ統計
    print("📊 ボラティリティ統計（年率%）:")
    vol_stats = rolling_vol.dropna().describe()
    print(vol_stats)
else:
    print("⚠️ ボラティリティ計算に十分なデータがありません。")

## 6. 移動平均線分析

In [ ]:
# 移動平均線の計算
if not price_pivot.empty:
    ma_periods = [5, 20, 50, 100]
    ma_data = {}

    for period in ma_periods:
        ma_data[f'MA_{period}'] = price_pivot.rolling(window=period).mean()

    # 最初の取引所の移動平均線チャート
    if len(price_pivot.columns) > 0:
        exchange_name = price_pivot.columns[0]
        
        fig = go.Figure()
        
        # 実際の価格
        fig.add_trace(
            go.Scatter(
                x=price_pivot.index,
                y=price_pivot[exchange_name],
                mode='lines',
                name=f'{exchange_name} Cash Price',
                line=dict(color='black', width=2)
            )
        )
        
        # 移動平均線
        colors = ['red', 'blue', 'green', 'purple']
        for i, period in enumerate(ma_periods):
            fig.add_trace(
                go.Scatter(
                    x=ma_data[f'MA_{period}'].index,
                    y=ma_data[f'MA_{period}'][exchange_name],
                    mode='lines',
                    name=f'MA {period}日',
                    line=dict(color=colors[i], width=1),
                    opacity=0.8
                )
            )
        
        fig.update_layout(
            title=f'🔸 {exchange_name}銅価格と移動平均線',
            xaxis_title='日付',
            yaxis_title='価格 (USD/MT)',
            height=500
        )
        
        fig.show()

        # 移動平均線からの乖離分析
        deviation = pd.DataFrame()
        for period in ma_periods:
            deviation[f'MA_{period}_deviation'] = (
                (price_pivot[exchange_name] - ma_data[f'MA_{period}'][exchange_name]) / ma_data[f'MA_{period}'][exchange_name] * 100
            )
        
        plt.figure(figsize=(14, 6))
        for col in deviation.columns:
            plt.plot(deviation.index, deviation[col], label=col.replace('_deviation', ''), alpha=0.7)
        
        plt.title('🔸 移動平均線からの乖離率（%）', fontsize=16, fontweight='bold')
        plt.xlabel('日付')
        plt.ylabel('乖離率 (%)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        plt.show()
        
        print("📊 移動平均線からの乖離統計:")
        print(deviation.describe())

## 7. 限月別価格分析

In [ ]:
# 限月別価格データの取得
if not copper_prices.empty:
    tenor_data = copper_prices[
        copper_prices['TenorTypeName'].isin(['Cash', '3M Futures', 'Generic 1st Future', 'Generic 2nd Future'])
    ].copy()

    # 限月別価格推移
    if not tenor_data.empty:
        fig = px.line(
            tenor_data, 
            x='TradeDate', 
            y='LastPrice', 
            color='TenorTypeName',
            title='🔸 銅価格 限月別推移',
            labels={'LastPrice': '価格 (USD/MT)', 'TradeDate': '日付'}
        )
        
        fig.update_layout(height=500)
        fig.show()

        # 過去30日間の限月別価格比較
        recent_data = tenor_data[tenor_data['TradeDate'] >= tenor_data['TradeDate'].max() - pd.Timedelta(days=30)]

        if not recent_data.empty:
            fig = px.box(
                recent_data, 
                x='TenorTypeName', 
                y='LastPrice', 
                color='ExchangeCode',
                title='🔸 過去30日間の限月別価格分布',
                labels={'LastPrice': '価格 (USD/MT)', 'TenorTypeName': '限月タイプ'}
            )

            fig.update_layout(height=500)
            fig.show()

            # 限月別価格統計
            print("📊 限月別価格統計（過去30日間）:")
            tenor_stats = recent_data.groupby(['ExchangeCode', 'TenorTypeName'])['LastPrice'].agg(['mean', 'std', 'min', 'max'])
            print(tenor_stats)

## 8. 分析サマリー

In [ ]:
# 分析結果のサマリー作成
print("📋 銅価格分析サマリー")
print("=" * 50)

if 'price_pivot' in locals() and not price_pivot.empty:
    # 最新価格
    latest_prices = price_pivot.iloc[-1]
    print("\n💰 最新価格 (USD/MT):")
    for exchange in latest_prices.index:
        print(f"  {exchange}: {latest_prices[exchange]:,.0f}")

    # 30日間の変動
    if len(price_pivot) >= 30:
        monthly_change = (price_pivot.iloc[-1] - price_pivot.iloc[-30]) / price_pivot.iloc[-30] * 100
        print("\n📈 30日間変動率 (%):")
        for exchange in monthly_change.index:
            print(f"  {exchange}: {monthly_change[exchange]:+.2f}%")

    # 最新ボラティリティ
    if 'rolling_vol' in locals() and not rolling_vol.empty:
        latest_vol = rolling_vol.iloc[-1]
        print("\n🌊 最新ボラティリティ (年率%):")
        for exchange in latest_vol.index:
            print(f"  {exchange}: {latest_vol[exchange]:.2f}%")

    # 相関分析結果
    if 'price_corr' in locals() and not price_corr.empty and len(price_corr.columns) > 1:
        print("\n🔗 取引所間相関（価格）:")
        exchanges = list(price_corr.columns)
        for i in range(len(exchanges)):
            for j in range(i+1, len(exchanges)):
                corr_val = price_corr.iloc[i, j]
                print(f"  {exchanges[i]} vs {exchanges[j]}: {corr_val:.3f}")

# データ品質
if 'copper_prices' in locals() and not copper_prices.empty:
    print("\n📊 データ品質:")
    print(f"  総レコード数: {len(copper_prices):,}")
    print(f"  取引所数: {len(copper_prices['ExchangeCode'].unique())}")
    print(f"  限月タイプ数: {len(copper_prices['TenorTypeName'].unique())}")
    print(f"  欠損値: {copper_prices['LastPrice'].isna().sum()}")
    print(f"  データ期間: {(copper_prices['TradeDate'].max() - copper_prices['TradeDate'].min()).days} 日間")

print("\n✅ 分析完了")
print("=" * 50)